# Goal
'Cleaner' code, doesn't work quite well

## Definitions

In [1]:
from dataclasses import dataclass

%matplotlib inline
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

@dataclass
class CFDResult:
    all_u: np.array
    simu: CFDSimulation

    def plot_velocities(self):
        def update(step = (0, self.all_u.shape[0], 1)):
            plt.imshow(np.linalg.norm(self.all_u[step], axis=2))
            plt.show()
        return interact(update)

    def plot_divergences(self):
        def update(step = (0, self.all_u.shape[0], 1)):
            div = simu.compute_divergence(self.all_u[step])
            plt.imshow(div)
            plt.colorbar()
            plt.show()
        return interact(update)

@dataclass
class CFDSimulation:
    steps = 15
    dt = 0.1
    grid_size = (1, 1)
    N = 32
    M = 32
    dn = grid_size[0] / N
    dm = grid_size[1] / M
    dim = (N, M, 2)
    initial_u = np.random.sample(size=dim)
    nu = 1
    rho = 1
    p = np.zeros(dim)

    def compute_convection(self, u_in):
        convection = np.zeros_like(u_in)
    
        # Adapt this function if you use a mesh with non uniform spacing
        def distance_1d(loc1, loc2):
            return sum(abs(loc1 - loc2) * (self.grid_size[0] / self.N, self.grid_size[1] / self.M))
        
        def derivative(loc, axis, component):
            prev_pt = np.clip(loc - axis, 0, [self.N - 1, self.M - 1])
            next_pt = np.clip(loc + axis, 0, [self.N - 1, self.M - 1])
            return (u_in[*next_pt, component] - u_in[*prev_pt, component]) / distance_1d(next_pt, prev_pt)
            
        
        for n in range(self.N):
            for m in range(self.M):
                grads = np.array([
                    [derivative(np.array([n, m]), np.array([1, 0]), 0), derivative(np.array([n, m]), np.array([1, 0]), 1)], 
                    [derivative(np.array([n, m]), np.array([0, 1]), 0), derivative(np.array([n, m]), np.array([0, 1]), 1)]])
                convection[n, m] = np.dot(u_in[n, m], grads)
                
        return convection

    def compute_viscous_drag(self, u_in):
        viscous_drag = np.zeros_like(u_in)
        
        def distance_1d(loc1, loc2):
            return sum(abs(loc1 - loc2) * (self.grid_size[0] / self.N, self.grid_size[1] / self.M))
            
        def derivative_2nd(loc, axis, component):
            prev_pt = np.clip(loc - axis, [0, 0], [self.N - 1, self.M - 1])
            next_pt = np.clip(loc + axis, [0, 0], [self.N - 1, self.M - 1])
            return (u_in[*next_pt, component] - (2 * u_in[*loc, component]) - u_in[*prev_pt, component]) / distance_1d(next_pt, prev_pt) ** 2
        
        for n in range(self.N):
            for m in range(self.M):
                for component in range(2):
                    viscous_drag[n, m, component] = derivative_2nd(np.array([n, m]), np.array([2, 0]), component) + derivative_2nd(np.array([n, m]), np.array([0, 2]), component)
        return viscous_drag * self.nu
        
                # (N,M,1) -> (N,M,2)
    def compute_pressure(self, pressure_in):
        pressure_out = np.zeros_like(self.initial_u)
        
        def distance_1d(loc1, loc2):
            return sum(abs(loc1 - loc2) * (1 / self.N, 1 / self.M))
        
        def derivative(loc, axis):
            prev_pt = np.clip(loc - axis, 0, [self.N - 1, self.M - 1])
            next_pt = np.clip(loc + axis, 0, [self.N - 1, self.M - 1])
            return (pressure_in[*next_pt, 0] - pressure_in[*prev_pt, 0]) / distance_1d(next_pt, prev_pt)
            
        for n in range(self.N):
            for m in range(self.M):
                pressure_out[n, m, 0] = derivative(np.array([n, m]), np.array([1, 0]))
                pressure_out[n, m, 1] = derivative(np.array([n, m]), np.array([0, 1]))
        
        return (1 / self.rho) * pressure_out

    def compute_updated_pressure(self, p_in, inter_u_in):
        p2 = np.zeros_like(p_in)
        
        def fetch(source, mock):
            def fetch_intern(n, m):
                if n < 0 or n >= self.N:
                    return mock
                if m < 0 or m >= self.M:
                    return mock
                return source[n, m]
            return fetch_intern
        
        fetch_p = fetch(p_in, 0)
        fetch_u = fetch(inter_u_in, 0)
        
        for n in range(self.N):
            for m in range(self.M):
                rnm = self.rho / self.dt * ((fetch_u(n + 1, m) - fetch_u(n - 1, m)) / (2 * self.dn) + (fetch_u(n, m + 1) - fetch_u(n, m - 1)) / (2 * self.dm) )
                x = (fetch_p(n + 2, m) - fetch_p(n - 2, m)) * 4 * self.dm ** 2 + (fetch_p(n, m + 2) - fetch_p(n, m - 2)) * 4 * self.dn ** 2
                x -= rnm * 16 * self.dn ** 2 * self.dm ** 2
                x /= 8 * (self.dn ** 2 + self.dm ** 2)
                p2[n, m] = x
    
        return self.compute_pressure(p2)

    def compute_divergence(self, u_in):
        divergence = np.zeros((self.N, self.M))
        bc = 0
        def fetch(n, m, c):
            if n < 0 or n >= self.N or m < 0 or m >= self.M:
                return bc
            return u_in[n, m, c]
            
        for n in range(self.N):
            for m in range(self.M):
                divergence[n, m] = (fetch(n + 1, m, 0) - fetch(n - 1, m, 0)) / (2 * self.dn) + (fetch(n, m + 1, 1) - fetch(n, m - 1, 1)) / (2 * self.dn)
        return divergence

    def launch(self):
        all_u = np.zeros((self.steps, self.N, self.M, 2))
        all_u[0] = self.initial_u
        for step in tqdm(range(1, self.steps)):
            u_update = -self.compute_convection(all_u[step - 1]) + self.compute_viscous_drag(all_u[step - 1])
            inter_u = u_update * self.dt
            self.p = self.compute_updated_pressure(self.p, inter_u)
            all_u[step] = all_u[step - 1] - self.p
        return CFDResult(all_u, self)

NameError: name 'CFDSimulation' is not defined

## Runs

In [ ]:

simu = CFDSimulation()
simu.dt = 0.001
result = simu.launch()
result.plot_velocities()

In [ ]:
result.plot_divergences()